In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
! pip3 install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, models, util, SentencesDataset, InputExample, losses, evaluation
from torch import nn
from torch.utils.data import DataLoader
import transformers
import os

In [ ]:
word_embedding_model = models.Transformer('bert-base-chinese', max_seq_length=64)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

In [ ]:
word_embedding_model.get_word_embedding_dimension()

In [ ]:
import pandas as pd
import numpy as np

data_path='/content/drive/My Drive/Sentence_pair_modeling/LCQMC/data/'
train=pd.read_csv(os.path.join(data_path,'train.tsv'),sep='\t',names=['s1','s2','label'])
train_examples = []
for i in range(len(train)):
  train_examples.append(InputExample(texts=[train.s1[i],train.s2[i]], label=int(train.label[i])))

In [ ]:
len(dev)

In [ ]:
dev=pd.read_csv(os.path.join(data_path,'LCQMC_dev.tsv'),sep='\t',names=['s1','s2','similarity'])
dev_examples = []
for i in range(len(dev)):
  dev_examples.append(InputExample(texts=[dev.s1[i],dev.s2[i]], label=int(dev.similarity[i])))

In [ ]:
# Define your train dataset, the dataloader and the train loss

train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
dev_dataset = SentencesDataset(dev_examples, model)
dev_dataloader = DataLoader(dev_dataset, shuffle=False, batch_size=300)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels = 2)
evaluator = evaluation.LabelAccuracyEvaluator(dev_dataloader,softmax_model = train_loss)

In [ ]:
model_save_path = '/content/drive/My Drive/Sentence_pair_modeling/Models/Sentence-BERT/output_sbert'
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3,  evaluator=evaluator, 
                  evaluation_steps=500, output_path = model_save_path)

In [ ]:
import torch
torch.save(train_loss, os.path.join(model_save_path,"3_Softmax/pytorch_model.bin"))

In [ ]:
import matplotlib.pyplot as plt
dev_result=pd.read_csv(os.path.join(model_save_path,'accuracy_evaluation_results.csv'))
dev_result.head()

In [ ]:
dev_result['steps'] = dev_result['steps'].apply(lambda x: x if x!=-1 else 7462)

In [ ]:
scale_ls = range(len(dev_result))
index_ls = [dev_result.epoch[i].astype(str) + '-' + dev_result.steps[i].astype(str) for i in range(len(dev_result))]
plt.figure(figsize=(16,7))
plt.plot(scale_ls[:15], dev_result.accuracy[:15], linewidth=3,label='epoch_0',color='r')
plt.plot(scale_ls[14:30], dev_result.accuracy[14:30], linewidth=3,label='epoch_1',color='orange')
plt.plot(scale_ls[29:], dev_result.accuracy[29:], linewidth=3,label='epoch_2',color='b')
plt.xticks(scale_ls,index_ls, rotation = 90) 
plt.ylabel('Accuracy',fontsize=12)
plt.xlabel('epoch-step',fontsize=12)
plt.title('Accuracy on dev dataset per 500 steps',fontsize=16)
plt.axvline(x=14,ls="--",c="k",alpha=0.7)
plt.axvline(x=29,ls="--",c="k",alpha=0.7)
plt.legend(loc='lower right')